In [1]:
import os
from io import open

# set directory for text input files
text_files = os.path.join(os.path.dirname("__file__"), 'texts')
files = os.listdir(text_files)

In [2]:
texts = []
authors = []

# Loop through each text file and append relevant data to lists
for doc in os.listdir(text_files):

    text = open(text_files + '\\' + doc, 'r', encoding='utf-8')
    # open the file, store as lower case
    text = text.read().lower()
    texts.append(text)
    
    authors.append(doc)



In [3]:
n_grams = set([])
treated_texts = []

import regex as re

for text in texts:
    
    # Remove high frequency stop words, as in Sims/Wu et. al
    stopwords = ['and', 'the', 'a', 'an']
    #text = text.split()
    
    # removes all punctuation, spaces and new lines    
    text = re.sub(ur"\p{P}+", "", text)
    
    # Split text, remove stopwords and rejoin string without spaces or newlines
    text = text.split()

    text = [x for x in text if x not in stopwords]
    
    text = ''.join(text)   
    
    #produces 6 character n-grams in a 'sliding window' across the text
    text = [text[i:i+9] for i in range(0, len(text), 1)]
    treated_texts.append(text)
    #n_gram_frequency = Counter(text)

    # Append features to the ngram list
    from collections import Counter
    c = Counter(text)

    # Append features to the ngram list
    for key, val in c.iteritems():
        if val > 1:
            n_grams.add(key)

In [4]:
from collections import OrderedDict

n_grams = OrderedDict.fromkeys(n_grams, 0.0) 

frequencies = []

t = []

for text in treated_texts:
    c = Counter(text)
    temp = OrderedDict(n_grams)
    
    for key, val in c.iteritems():
        if val > 1:
            temp[key] = float(val)

    frequencies.append(temp.values())
 

In [5]:
from scipy.stats import entropy
from numpy.linalg import norm
import numpy as np

def JSD(P, Q):
    P = P / norm(np.array(P), ord=1)
    Q = Q / norm(np.array(Q), ord=1)
    M = 0.5 * (P + Q)
    jsd = (0.5 * entropy(P, M)) + (0.5 * entropy(Q, M))
    return jsd

In [6]:
#TODO - make this more efficient

distance_matrix = [[] for x in range(len(frequencies))]

for i in range(len(frequencies)):
    for j in range(len(frequencies)):
        
        if i==j:
            distance_matrix[i].append(0)
            break
        
        distance_matrix[i].append(JSD(frequencies[i], frequencies[j]))  

In [7]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio.Phylo.TreeConstruction import _DistanceMatrix 
from Bio import Phylo

m = _DistanceMatrix(authors, distance_matrix)


constructor = DistanceTreeConstructor()

njtree = constructor.nj(m)


Phylo.draw_ascii(njtree)



            _____________________________ conrad 3.txt
          ,|
          ||____________________________ conrad 2.txt
         _|
        | |  _____________________________ twain 4.txt
        | | |
        | |_|   _____________________________ twain 2.txt
        |   | ,|
        |   | ||_________________________________ twain 1.txt
        |   |_|
      __|     |     ________________________________ shakespeare 1.txt
     |  |     |____|
     |  |          |__________________________ conrad 1.txt
     |  |
     |  |            _________________________________ shakespeare 3.txt
     |  |    _______|
     |  |   |       |_______________________________ shakespeare 2.txt
    ,|  |___|
    ||      |   __________________________________________ B-List 1.txt
    ||      | _|
    ||      || |_________________________________________ A-List 1.txt
    ||       |
    ||       |_________________________________________ all burns.txt
    ||
  __||    ____________________________ austen 2.tx

In [9]:
# # PCA using the ngram frequencies, just for laughs

# import numpy as np

# ###### Convert both X and Y to numpy arrays
# X = np.array(frequencies)
# y = np.array(authors)
# markers = set(authors)

# # Standardise data, perform PCA
# from sklearn.decomposition import PCA as sklearnPCA
# from sklearn.preprocessing import StandardScaler

# X_std = StandardScaler().fit_transform(X)
# sklearn_pca = sklearnPCA(n_components=2)
# Y_sklearn = sklearn_pca.fit_transform(X_std)

# import plotly.plotly as py
# from plotly.graph_objs import *
# import plotly.tools as tls

# traces = []

# for author in markers:

#     trace = Scatter(
#         x=Y_sklearn[y==author,0],
#         y=Y_sklearn[y==author,1],
#         mode='markers',
#         name=author,
#         marker=Marker(
#             size=14))
#     traces.append(trace)


# data = Data(traces)
# layout = Layout(title='PCA of Burns and contemporaries',
#                 xaxis=XAxis(title='Principal Component 1'),
#                 yaxis=YAxis(title='Principal Component 2'))
# fig = Figure(data=data, layout=layout)
# py.iplot(fig)